In [2]:
import java.util.concurrent._
import language.implicitConversions

scala.language$@33c4468

In [3]:
// id : ex_7.1

class Par[A](ap : () => A) {
    
    val a : () => A = ap
        
    def get : A = a()
    
}

object Par {
    
    def unit[A](a : => A) : Par[A] = new Par(() => a)

    def map2[A,B,C](pa : Par[A], pb: Par[B])(f: (A, B) => C): Par[C] = {
        unit(f(pa.get,pb.get))
    }
    
}


$line20.$read$$iw$$iw$Par$@349fa2a5

In [4]:
def sum(ints: IndexedSeq[Int]): Par[Int] = 
    if (ints.size <= 1) Par.unit(ints.headOption getOrElse 0) 
    else {
        val (l,r) = ints.splitAt(ints.length/2)
        Par.map2(sum(l), sum(r))(_ + _)
    }

sum: (ints: IndexedSeq[Int])Par[Int]


In [5]:
sum(IndexedSeq(1,2,3,4,5)).get

15

In [8]:
// id : ex_7.2

// Shamelessly (or shamefully) taking the relevant portions from FP in Scala answers
type Par[A] = ExecutorService => Future[A]

object Par {

  def run[A](s: ExecutorService)(a: Par[A]): Future[A] = a(s)

  def unit[A](a: A): Par[A] = (es: ExecutorService) => UnitFuture(a) // `unit` is represented as a function that returns a `UnitFuture`, which is a simple implementation of `Future` that just wraps a constant value. It doesn't use the `ExecutorService` at all. It's always done and can't be cancelled. Its `get` method simply returns the value that we gave it.

  private case class UnitFuture[A](get: A) extends Future[A] {
    def isDone = true
    def get(timeout: Long, units: TimeUnit) = get
    def isCancelled = false
    def cancel(evenIfRunning: Boolean): Boolean = false
  }

  def map2[A,B,C](a: Par[A], b: Par[B])(f: (A,B) => C): Par[C] = // `map2` doesn't evaluate the call to `f` in a separate logical thread, in accord with our design choice of having `fork` be the sole function in the API for controlling parallelism. We can always do `fork(map2(a,b)(f))` if we want the evaluation of `f` to occur in a separate thread.
    (es: ExecutorService) => {
      val af = a(es)
      val bf = b(es)
      UnitFuture(f(af.get, bf.get)) // This implementation of `map2` does _not_ respect timeouts. It simply passes the `ExecutorService` on to both `Par` values, waits for the results of the Futures `af` and `bf`, applies `f` to them, and wraps them in a `UnitFuture`. In order to respect timeouts, we'd need a new `Future` implementation that records the amount of time spent evaluating `af`, then subtracts that time from the available time allocated for evaluating `bf`.
    }

  def fork[A](a: => Par[A]): Par[A] = // This is the simplest and most natural implementation of `fork`, but there are some problems with it--for one, the outer `Callable` will block waiting for the "inner" task to complete. Since this blocking occupies a thread in our thread pool, or whatever resource backs the `ExecutorService`, this implies that we're losing out on some potential parallelism. Essentially, we're using two threads when one should suffice. This is a symptom of a more serious problem with the implementation, and we will discuss this later in the chapter.
    es => es.submit(new Callable[A] {
      def call = a(es).get
    })

  def lazyUnit[A](a: => A): Par[A] = fork(unit(a))

}


$line25.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$Par$@5326d231

In [15]:
// id : try esx_7.2

def sum(ints: IndexedSeq[Int]): Par[Int] = 
    if (ints.size <= 1) Par.unit(ints.headOption getOrElse 0) 
    else {
        val (l,r) = ints.splitAt(ints.length/2)
        Par.map2(sum(l), sum(r))(_ + _)
    }

val future = Par.run(Executors.newFixedThreadPool(4))(sum(IndexedSeq(1,2,3,4,5)))

println("future", future)
println("get", future.get)

(future,UnitFuture(15))
(get,15)


null